In [1]:
# make sure changes in code take place immediately
%load_ext autoreload
%autoreload 2
import pandas as pd # occasionally this would get left out
    
# load the chapyter extension for Jupyter Notebook
%load_ext chapyter

#Define the notebook name as a variable
import os
os.environ["NOTEBOOK_NAME"] = "Context.ipynb"


In [2]:
%%mimicSQL 

I'd like to study if there is an association between sex and in-hospital mortality in patients with sepsis admitted to the ICU.
Can you get me all the patients admitted to an ICU with sepsis, severe sepsis, or septic shock that are 18 or older, as well as their admit times, discharge time, and time of death? 
I'd also like to add a few variables – can you include the following variables in the data frame? 
- ethnicity
- the first ICU service the patient was admitted to
- admission type

Remember that every column in the SELECT clause must be part of an aggregate function or listed in the GROUP BY clause. 

Context:  ============================================================
**Clinical Researcher:** %%mimicSQL  --- I'd like to study if there is an association between sex and in-hospital mortality in patients with sepsis admitted to the ICU.
Can you get me all the patients admitted to an ICU with sepsis, severe sepsis, or septic shock that are 18 or older, as well as their admit times, discharge time, and time of death? 
I'd also like to add a few variables – can you include the following variables in the data frame? 
- ethnicity
- the first ICU service the patient was admitted to
- admission type --- Remember that every column in the SELECT clause must be part of an aggregate function or listed in the GROUP BY clause. 

**AI Research Assistant:**
Sure, I can help with that. We will need to join several tables to get all the information you need. Here is the SQL query:

This query will return a list of patients who were admitted to the ICU with sepsis, severe sepsis, or septic shock, and

In [4]:
%%runSQL 


SELECT 
    p.subject_id,
    a.hadm_id,
    a.admittime,
    a.dischtime,
    p.dod,
    p.gender,
    p.dob,
    a.ethnicity,
    a.admission_type,
    ie.first_careunit
FROM 
    patients p
JOIN 
    admissions a ON p.subject_id = a.subject_id
JOIN 
    icustays ie ON a.hadm_id = ie.hadm_id
JOIN 
    diagnoses_icd d ON a.hadm_id = d.hadm_id
WHERE 
    d.icd9_code IN ('99591', '99592', '78552') -- ICD-9 codes for sepsis, severe sepsis, and septic shock
    AND DATE_DIFF('year', p.dob, a.admittime) >= 18 -- Patients 18 or older
GROUP BY 
    p.subject_id,
    a.hadm_id,
    a.admittime,
    a.dischtime,
    p.dod,
    p.gender,
    p.dob,
    a.ethnicity,
    a.admission_type,
    ie.first_careunit


,subject_id,hadm_id,admittime,dischtime,dod,gender,dob,ethnicity,admission_type,first_careunit
0,29503,110600,2153-05-23 20:04:00.000,2144-02-21 13:30:00.000,2111-04-07 00:00:00.000,M,2118-10-27 00:00:00.000,PATIENT DECLINED TO ANSWER,URGENT,CCU
1,96632,118662,2116-03-04 15:59:00.000,2168-05-03 12:00:00.000,2133-07-01 00:00:00.000,F,2053-01-19 00:00:00.000,ASIAN - ASIAN INDIAN,URGENT,MICU
2,53399,157942,2188-11-30 23:17:00.000,2127-07-08 04:30:00.000,2169-04-21 00:00:00.000,F,2048-07-11 00:00:00.000,BLACK/AFRICAN AMERICAN,EMERGENCY,CCU
3,87880,109485,2156-11-11 19:57:00.000,2173-06-04 13:44:00.000,2108-06-03 00:00:00.000,F,2080-01-30 00:00:00.000,OTHER,ELECTIVE,TSICU
4,31135,111413,2125-05-05 15:42:00.000,2149-07-17 14:45:00.000,2179-11-30 00:00:00.000,F,1898-01-06 00:00:00.000,WHITE - OTHER EUROPEAN,URGENT,MICU


In [3]:
df.head()

,subject_id,hadm_id,admittime,dischtime,dod,gender,dob,ethnicity,admission_type,first_careunit
0,703,124912,2106-08-06 17:09:00.000,2106-08-08 11:50:00.000,None,M,2032-03-08 00:00:00.000,WHITE,EMERGENCY,MICU
1,781,163526,2117-09-21 23:21:00.000,2117-11-27 10:00:00.000,2118-01-21 00:00:00.000,F,2041-08-18 00:00:00.000,WHITE,EMERGENCY,MICU
2,788,139716,2194-01-13 20:48:00.000,2194-01-18 14:30:00.000,None,F,2122-07-19 00:00:00.000,WHITE,EMERGENCY,MICU
3,797,100863,2139-08-24 22:00:00.000,2139-08-29 14:45:00.000,2141-03-28 00:00:00.000,M,2096-11-25 00:00:00.000,WHITE,EMERGENCY,MICU
4,822,191344,2176-09-28 17:41:00.000,2176-10-08 16:00:00.000,None,M,2145-10-30 00:00:00.000,WHITE,EMERGENCY,MICU


In [25]:
%%runSQL 


SELECT 
    p.subject_id,
    p.gender,
    p.dob,
    a.hadm_id,
    a.admittime,
    a.dischtime,
    a.deathtime,
    a.admission_type,
    a.ethnicity,
    i.first_careunit
FROM 
    patients p
JOIN 
    admissions a ON p.subject_id = a.subject_id
JOIN 
    icustays i ON a.hadm_id = i.hadm_id
JOIN 
    diagnoses_icd d ON a.hadm_id = d.hadm_id
WHERE 
    d.icd9_code IN ('99591', '99592', '78552')
    AND DATE_DIFF('year', p.dob, a.admittime) >= 18


,subject_id,gender,dob,hadm_id,admittime,dischtime,deathtime,admission_type,ethnicity,first_careunit
0,66876,M,2057-06-23 00:00:00.000,122645,2133-02-09 14:00:00.000,2133-02-20 13:38:00.000,2133-02-20 13:38:00.000,ELECTIVE,WHITE,CSRU


In [16]:
%%mimicSQL 

Great. Now I'd like to get the same patients, but only select their first ICU admission in the database. 

Context:  ============================================================
**Clinical Researcher:** %%mimicSQL  --- I'd like to study if there is an association between sex and in-hospital mortality in patients with sepsis admitted to the ICU.
Can you get me all the patients admitted to an ICU with sepsis, severe sepsis, or septic shock that are 18 or older, as well as their admit times, discharge time, and time of death? 
I'd also like to add a few variables – can you include the following variables in the data frame? 
- ethnicity
- the first ICU service the patient was admitted to
- admission type --- Remember that every column in the SELECT clause must be part of an aggregate function or listed in the GROUP BY clause. 

**AI Research Assistant:** Context:  ============================================================
**Clinical Researcher:** %%mimicSQL  --- I'd like to study if there is an association between sex and in-hospital mortality in patients with sepsis admitted to the ICU.
Can 

In [20]:
%%runSQL 


WITH first_icu_admission AS (
    SELECT 
        p.subject_id,
        p.gender,
        p.dob,
        a.hadm_id,
        a.admittime,
        a.dischtime,
        a.deathtime,
        a.admission_type,
        a.ethnicity,
        i.first_careunit,
        ROW_NUMBER() OVER(PARTITION BY p.subject_id ORDER BY i.intime) as icu_admission_rank
    FROM 
        patients p
    JOIN 
        admissions a ON p.subject_id = a.subject_id
    JOIN 
        icustays i ON a.hadm_id = i.hadm_id
    JOIN 
        diagnoses_icd d ON a.hadm_id = d.hadm_id
    WHERE 
        d.icd9_code IN ('99591', '99592', '78552')
        AND DATE_DIFF('year', p.dob, a.admittime) >= 18
)
SELECT *
FROM first_icu_admission
WHERE icu_admission_rank = 1


,subject_id,gender,dob,hadm_id,admittime,dischtime,deathtime,admission_type,ethnicity,first_careunit,icu_admission_rank
0,66894,F,2069-03-07 00:00:00.000,157688,2126-08-10 21:10:00.000,2126-08-11 19:29:00.000,2126-08-11 19:29:00.000,EMERGENCY,HISPANIC OR LATINO,MICU,1


In [21]:
%%mimicSQL
Perfect. Now can we add two things to that data frame (without changing the number of entries): 
- the patient's first lactate of that ICU admission
- the patient's first creatinine of that ICU admission
The number of rows should stay the same, we simply want to add those two lab values to the existing table. 

Context:  ============================================================
**Clinical Researcher:** %%mimicSQL  --- I'd like to study if there is an association between sex and in-hospital mortality in patients with sepsis admitted to the ICU.
Can you get me all the patients admitted to an ICU with sepsis, severe sepsis, or septic shock that are 18 or older, as well as their admit times, discharge time, and time of death? 
I'd also like to add a few variables – can you include the following variables in the data frame? 
- ethnicity
- the first ICU service the patient was admitted to
- admission type --- Remember that every column in the SELECT clause must be part of an aggregate function or listed in the GROUP BY clause. 

**AI Research Assistant:** Context:  ============================================================
**Clinical Researcher:** %%mimicSQL  --- I'd like to study if there is an association between sex and in-hospital mortality in patients with sepsis admitted to the ICU.
Can 

In [24]:
%%runSQL 


WITH first_icu_admission AS (
    SELECT 
        p.subject_id,
        p.gender,
        p.dob,
        a.hadm_id,
        a.admittime,
        a.dischtime,
        a.deathtime,
        a.admission_type,
        a.ethnicity,
        i.first_careunit,
        i.intime,
        i.outtime,
        ROW_NUMBER() OVER(PARTITION BY p.subject_id ORDER BY i.intime) as icu_admission_rank
    FROM 
        patients p
    JOIN 
        admissions a ON p.subject_id = a.subject_id
    JOIN 
        icustays i ON a.hadm_id = i.hadm_id
    JOIN 
        diagnoses_icd d ON a.hadm_id = d.hadm_id
    WHERE 
        d.icd9_code IN ('99591', '99592', '78552')
        AND DATE_DIFF('year', p.dob, a.admittime) >= 18
),
first_lactate AS (
    SELECT 
        le.subject_id,
        le.hadm_id,
        le.valuenum as first_lactate,
        ROW_NUMBER() OVER(PARTITION BY le.subject_id, le.hadm_id ORDER BY le.charttime) as lactate_rank
    FROM 
        labevents le
    WHERE 
        le.itemid = 50813
        AND le.subject_id IN (SELECT subject_id FROM first_icu_admission)
        AND le.hadm_id IN (SELECT hadm_id FROM first_icu_admission)
),
first_creatinine AS (
    SELECT 
        le.subject_id,
        le.hadm_id,
        le.valuenum as first_creatinine,
        ROW_NUMBER() OVER(PARTITION BY le.subject_id, le.hadm_id ORDER BY le.charttime) as creatinine_rank
    FROM 
        labevents le
    WHERE 
        le.itemid = 50912
        AND le.subject_id IN (SELECT subject_id FROM first_icu_admission)
        AND le.hadm_id IN (SELECT hadm_id FROM first_icu_admission)
)
SELECT 
    fia.*,
    fl.first_lactate,
    fc.first_creatinine
FROM 
    first_icu_admission fia
LEFT JOIN 
    first_lactate fl ON fia.subject_id = fl.subject_id AND fia.hadm_id = fl.hadm_id AND fl.lactate_rank = 1
LEFT JOIN 
    first_creatinine fc ON fia.subject_id = fc.subject_id AND fia.hadm_id = fc.hadm_id AND fc.creatinine_rank = 1
WHERE 
    fia.icu_admission_rank = 1


,subject_id,gender,dob,hadm_id,admittime,dischtime,deathtime,admission_type,ethnicity,first_careunit,intime,outtime,icu_admission_rank,first_lactate,first_creatinine
0,804,M,2085-05-01 00:00:00.000,138064,2154-08-26 11:29:00.000,2154-09-02 12:00:00.000,2154-09-02 12:00:00.000,EMERGENCY,WHITE,MICU,2154-09-01 05:44:36.000,2154-09-02 20:55:20.000,1,3.5,1.3


In [19]:
%%runSQL 


WITH first_icu_admission AS (
    SELECT 
        p.subject_id,
        p.gender,
        p.dob,
        a.hadm_id,
        a.admittime,
        a.dischtime,
        a.deathtime,
        a.admission_type,
        a.ethnicity,
        i.first_careunit,
        i.icustay_id,
        ROW_NUMBER() OVER(PARTITION BY p.subject_id ORDER BY i.intime) as icu_admission_rank
    FROM 
        patients p
    JOIN 
        admissions a ON p.subject_id = a.subject_id
    JOIN 
        icustays i ON a.hadm_id = i.hadm_id
    JOIN 
        diagnoses_icd d ON a.hadm_id = d.hadm_id
    WHERE 
        d.icd9_code IN ('99591', '99592', '78552')
        AND DATE_DIFF('year', p.dob, a.admittime) >= 18
),
first_lactate AS (
    SELECT 
        le.subject_id,
        le.hadm_id,
        le.icustay_id,
        le.valuenum as first_lactate,
        ROW_NUMBER() OVER(PARTITION BY le.subject_id, le.icustay_id ORDER BY le.charttime) as lactate_rank
    FROM 
        labevents le
    WHERE 
        le.itemid = 50813
),
first_creatinine AS (
    SELECT 
        le.subject_id,
        le.hadm_id,
        le.icustay_id,
        le.valuenum as first_creatinine,
        ROW_NUMBER() OVER(PARTITION BY le.subject_id, le.icustay_id ORDER BY le.charttime) as creatinine_rank
    FROM 
        labevents le
    WHERE 
        le.itemid = 50912
)
SELECT 
    fia.*,
    fl.first_lactate,
    fc.first_creatinine
FROM 
    first_icu_admission fia
LEFT JOIN 
    first_lactate fl ON fia.subject_id = fl.subject_id AND fia.icustay_id = fl.icustay_id AND fl.lactate_rank = 1
LEFT JOIN 
    first_creatinine fc ON fia.subject_id = fc.subject_id AND fia.icustay_id = fc.icustay_id AND fc.creatinine_rank = 1
WHERE 
    fia.icu_admission_rank = 1


Query failed! State: FAILED
Reason: COLUMN_NOT_FOUND: line 31:9: Column 'le.icustay_id' cannot be resolved or requester is not authorized to access requested resources


AttributeError: 'bool' object has no attribute 'head'

In [23]:
%%mimicSQL
With the above code, I get the error: Query failed! State: FAILED
Reason: COLUMN_NOT_FOUND: line 31:9: Column 'le.icustay_id' cannot be resolved or requester is not authorized to access requested resources
Can you produce new SQL code that fixes that? 

Context:  ============================================================
**Clinical Researcher:** %%mimicSQL  --- I'd like to study if there is an association between sex and in-hospital mortality in patients with sepsis admitted to the ICU.
Can you get me all the patients admitted to an ICU with sepsis, severe sepsis, or septic shock that are 18 or older, as well as their admit times, discharge time, and time of death? 
I'd also like to add a few variables – can you include the following variables in the data frame? 
- ethnicity
- the first ICU service the patient was admitted to
- admission type --- Remember that every column in the SELECT clause must be part of an aggregate function or listed in the GROUP BY clause. 

**AI Research Assistant:** Context:  ============================================================
**Clinical Researcher:** %%mimicSQL  --- I'd like to study if there is an association between sex and in-hospital mortality in patients with sepsis admitted to the ICU.
Can 

InvalidRequestError: This model's maximum context length is 8192 tokens. However, you requested 8567 tokens (7567 in the messages, 1000 in the completion). Please reduce the length of the messages or completion.